In [1]:
# import pandas as pd
# import numpy as np
# import os
# import pickle
# from statsmodels.tsa.arima.model import ARIMA
# from statsmodels.tsa.stattools import adfuller
# from sklearn.metrics import mean_absolute_error
# import warnings
# warnings.filterwarnings("ignore")

# # Fungsi untuk memuat dan memproses data
# def load_and_preprocess_data(file_path):
#     data = pd.read_csv(file_path)
#     data['Date'] = pd.to_datetime(data['Date'])
#     data = data.dropna()
#     return data

# # Fungsi untuk membagi data berdasarkan waktu
# def split_data_by_time(student_data, train_days=24):
#     student_data = student_data.sort_values('Date')
#     unique_dates = student_data['Date'].unique()
#     train_dates = unique_dates[:train_days]
#     test_dates = unique_dates[train_days:]
#     train_data = student_data[student_data['Date'].isin(train_dates)]
#     test_data = student_data[student_data['Date'].isin(test_dates)]
#     return train_data, test_data

# # Fungsi untuk uji stasioneritas
# def adf_test(series):
#     result = adfuller(series.dropna())
#     return result[1] < 0.05  # True jika stasioner

# # Fungsi untuk melatih dan menyimpan model ARIMA
# def train_arima(student_data, student_id, model_dir="model/arima"):
#     train_data, test_data = split_data_by_time(student_data)
    
#     # Pastikan direktori model ada
#     os.makedirs(model_dir, exist_ok=True)
    
#     # Tuning ARIMA untuk Stress Level
#     best_mae = float('inf')
#     best_params = (1, 1, 1)
#     best_model_fit = None
    
#     if adf_test(train_data['Stress Level (GSR)']):
#         for p in range(3):
#             for d in range(2):
#                 for q in range(3):
#                     try:
#                         arima_model = ARIMA(train_data['Stress Level (GSR)'], order=(p, d, q))
#                         arima_fit = arima_model.fit()
#                         arima_pred = arima_fit.forecast(steps=len(test_data))
#                         mae = mean_absolute_error(test_data['Stress Level (GSR)'], arima_pred)
#                         if mae < best_mae:
#                             best_mae = mae
#                             best_params = (p, d, q)
#                             best_model_fit = arima_fit
#                     except:
#                         continue
    
#     # Simpan model terbaik
#     if best_model_fit:
#         model_path = os.path.join(model_dir, f"arima_student_{student_id}.pkl")
#         with open(model_path, 'wb') as f:
#             pickle.dump(best_model_fit, f)
#         print(f"ARIMA model for Student {student_id} saved to {model_path}")
    
#     return best_model_fit, best_params, best_mae

# # Fungsi utama
# def main():
#     data = load_and_preprocess_data('student_monnitoring_data.csv')
#     user_groups = data.groupby('Student ID')
    
#     # Proses hanya 5 mahasiswa sebagai contoh
#     for i, (student_id, student_data) in enumerate(user_groups):
#         if i >= 5:
#             break
#         print(f"\nTraining ARIMA for Student {student_id}")
#         model_fit, params, mae = train_arima(student_data, student_id)
#         print(f"Best ARIMA params: {params}, MAE: {mae:.4f}")



In [3]:
import pandas as pd
import numpy as np
import os
import pickle
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings("ignore")

# Fungsi untuk memuat dan memproses data
def load_and_preprocess_data(file_path):
    data = pd.read_csv(file_path)
    data['Date'] = pd.to_datetime(data['Date'])
    data = data.dropna()
    return data

# Fungsi untuk membagi data berdasarkan waktu
def split_data_by_time(student_data, train_days=24):
    student_data = student_data.sort_values('Date')
    unique_dates = student_data['Date'].unique()
    train_dates = unique_dates[:train_days]
    test_dates = unique_dates[train_days:]
    train_data = student_data[student_data['Date'].isin(train_dates)]
    test_data = student_data[student_data['Date'].isin(test_dates)]
    return train_data, test_data

# Fungsi untuk uji stasioneritas
def adf_test(series):
    result = adfuller(series.dropna())
    return result[1] < 0.05  # True jika stasioner

# Fungsi untuk menjadikan data stasioner dengan differencing maksimal 2 kali
def make_stationary(series, max_diff=2):
    diff_count = 0
    while not adf_test(series) and diff_count < max_diff:
        series = series.diff().dropna()
        diff_count += 1
    return series, diff_count

# Fungsi untuk melatih dan menyimpan model ARIMA
def train_arima(student_data, student_id, model_dir="model/arima"):
    train_data, test_data = split_data_by_time(student_data)
    
    # Pastikan direktori model ada
    os.makedirs(model_dir, exist_ok=True)

    original_train = train_data['Stress Level (GSR)'].copy()
    stationary_train, d_diff = make_stationary(original_train)

    best_mae = float('inf')
    best_rmse = float('inf')
    best_params = (1, d_diff, 1)
    best_model_fit = None
    
    for p in range(3):
        for q in range(3):
            try:
                model = ARIMA(original_train, order=(p, d_diff, q))
                model_fit = model.fit()
                forecast = model_fit.forecast(steps=len(test_data))
                mae = mean_absolute_error(test_data['Stress Level (GSR)'], forecast)
                rmse = np.sqrt(mean_squared_error(test_data['Stress Level (GSR)'], forecast))
                if mae < best_mae:
                    best_mae = mae
                    best_rmse = rmse
                    best_params = (p, d_diff, q)
                    best_model_fit = model_fit
            except:
                continue

    # Simpan model terbaik
    if best_model_fit:
        model_path = os.path.join(model_dir, f"arima_student_{student_id}.pkl")
        with open(model_path, 'wb') as f:
            pickle.dump(best_model_fit, f)
        print(f"ARIMA model for Student {student_id} saved to {model_path}")
    
    return best_model_fit, best_params, best_mae, best_rmse

# Fungsi utama
def main():
    data = load_and_preprocess_data('student_monnitoring_data.csv')
    user_groups = data.groupby('Student ID')
    
    for i, (student_id, student_data) in enumerate(user_groups):
        # if i >= 5:
        #     break
        print(f"\nTraining ARIMA for Student {student_id}")
        model_fit, params, mae, rmse = train_arima(student_data, student_id)
        print(f"Best ARIMA params: {params}, MAE: {mae:.4f}, RMSE: {rmse:.4f}")

In [4]:
if __name__ == "__main__":
    main()


Training ARIMA for Student 1
ARIMA model for Student 1 saved to model/arima\arima_student_1.pkl
Best ARIMA params: (0, 0, 1), MAE: 1.2130, RMSE: 1.4649

Training ARIMA for Student 2
ARIMA model for Student 2 saved to model/arima\arima_student_2.pkl
Best ARIMA params: (2, 0, 2), MAE: 1.2455, RMSE: 1.5445

Training ARIMA for Student 3
ARIMA model for Student 3 saved to model/arima\arima_student_3.pkl
Best ARIMA params: (0, 2, 2), MAE: 1.0776, RMSE: 1.3100

Training ARIMA for Student 4
ARIMA model for Student 4 saved to model/arima\arima_student_4.pkl
Best ARIMA params: (2, 0, 1), MAE: 1.6113, RMSE: 1.6737

Training ARIMA for Student 5
ARIMA model for Student 5 saved to model/arima\arima_student_5.pkl
Best ARIMA params: (1, 0, 0), MAE: 1.6149, RMSE: 1.7167

Training ARIMA for Student 6
ARIMA model for Student 6 saved to model/arima\arima_student_6.pkl
Best ARIMA params: (0, 0, 1), MAE: 1.1914, RMSE: 1.4196

Training ARIMA for Student 7
ARIMA model for Student 7 saved to model/arima\arima

ARIMA model for Student 54 saved to model/arima\arima_student_54.pkl
Best ARIMA params: (1, 0, 1), MAE: 0.8594, RMSE: 1.0205

Training ARIMA for Student 55
ARIMA model for Student 55 saved to model/arima\arima_student_55.pkl
Best ARIMA params: (2, 1, 2), MAE: 0.6543, RMSE: 0.9078

Training ARIMA for Student 56
ARIMA model for Student 56 saved to model/arima\arima_student_56.pkl
Best ARIMA params: (1, 2, 2), MAE: 1.0413, RMSE: 1.4434

Training ARIMA for Student 57
ARIMA model for Student 57 saved to model/arima\arima_student_57.pkl
Best ARIMA params: (2, 0, 2), MAE: 1.0661, RMSE: 1.2061

Training ARIMA for Student 58
ARIMA model for Student 58 saved to model/arima\arima_student_58.pkl
Best ARIMA params: (0, 0, 0), MAE: 1.1758, RMSE: 1.3047

Training ARIMA for Student 59
ARIMA model for Student 59 saved to model/arima\arima_student_59.pkl
Best ARIMA params: (0, 0, 1), MAE: 1.5221, RMSE: 1.5900

Training ARIMA for Student 60
ARIMA model for Student 60 saved to model/arima\arima_student_60

ARIMA model for Student 107 saved to model/arima\arima_student_107.pkl
Best ARIMA params: (0, 0, 1), MAE: 1.0382, RMSE: 1.1006

Training ARIMA for Student 108
ARIMA model for Student 108 saved to model/arima\arima_student_108.pkl
Best ARIMA params: (2, 0, 2), MAE: 1.0140, RMSE: 1.2000

Training ARIMA for Student 109
ARIMA model for Student 109 saved to model/arima\arima_student_109.pkl
Best ARIMA params: (2, 0, 2), MAE: 1.1456, RMSE: 1.3371

Training ARIMA for Student 110
ARIMA model for Student 110 saved to model/arima\arima_student_110.pkl
Best ARIMA params: (0, 0, 0), MAE: 1.2867, RMSE: 1.5234

Training ARIMA for Student 111
ARIMA model for Student 111 saved to model/arima\arima_student_111.pkl
Best ARIMA params: (0, 0, 2), MAE: 0.6914, RMSE: 0.8920

Training ARIMA for Student 112
ARIMA model for Student 112 saved to model/arima\arima_student_112.pkl
Best ARIMA params: (1, 2, 0), MAE: 0.8498, RMSE: 1.1818

Training ARIMA for Student 113
ARIMA model for Student 113 saved to model/ari

ARIMA model for Student 159 saved to model/arima\arima_student_159.pkl
Best ARIMA params: (1, 0, 2), MAE: 1.3366, RMSE: 1.4057

Training ARIMA for Student 160
ARIMA model for Student 160 saved to model/arima\arima_student_160.pkl
Best ARIMA params: (1, 2, 2), MAE: 0.9967, RMSE: 1.1491

Training ARIMA for Student 161
ARIMA model for Student 161 saved to model/arima\arima_student_161.pkl
Best ARIMA params: (0, 0, 2), MAE: 1.1068, RMSE: 1.2573

Training ARIMA for Student 162
ARIMA model for Student 162 saved to model/arima\arima_student_162.pkl
Best ARIMA params: (2, 0, 2), MAE: 0.9849, RMSE: 1.2235

Training ARIMA for Student 163
ARIMA model for Student 163 saved to model/arima\arima_student_163.pkl
Best ARIMA params: (1, 1, 2), MAE: 0.4928, RMSE: 0.5865

Training ARIMA for Student 164
ARIMA model for Student 164 saved to model/arima\arima_student_164.pkl
Best ARIMA params: (2, 0, 0), MAE: 1.0256, RMSE: 1.1484

Training ARIMA for Student 165
ARIMA model for Student 165 saved to model/ari

ARIMA model for Student 211 saved to model/arima\arima_student_211.pkl
Best ARIMA params: (0, 2, 2), MAE: 1.4240, RMSE: 1.5168

Training ARIMA for Student 212
ARIMA model for Student 212 saved to model/arima\arima_student_212.pkl
Best ARIMA params: (2, 0, 2), MAE: 0.6455, RMSE: 0.8157

Training ARIMA for Student 213
ARIMA model for Student 213 saved to model/arima\arima_student_213.pkl
Best ARIMA params: (2, 2, 1), MAE: 0.8996, RMSE: 1.1033

Training ARIMA for Student 214
ARIMA model for Student 214 saved to model/arima\arima_student_214.pkl
Best ARIMA params: (1, 2, 1), MAE: 1.0272, RMSE: 1.2014

Training ARIMA for Student 215
ARIMA model for Student 215 saved to model/arima\arima_student_215.pkl
Best ARIMA params: (0, 0, 0), MAE: 0.7642, RMSE: 0.9749

Training ARIMA for Student 216
ARIMA model for Student 216 saved to model/arima\arima_student_216.pkl
Best ARIMA params: (2, 0, 1), MAE: 1.1960, RMSE: 1.3459

Training ARIMA for Student 217
ARIMA model for Student 217 saved to model/ari

ARIMA model for Student 263 saved to model/arima\arima_student_263.pkl
Best ARIMA params: (1, 1, 0), MAE: 0.8726, RMSE: 1.0928

Training ARIMA for Student 264
ARIMA model for Student 264 saved to model/arima\arima_student_264.pkl
Best ARIMA params: (0, 0, 1), MAE: 1.1188, RMSE: 1.1932

Training ARIMA for Student 265
ARIMA model for Student 265 saved to model/arima\arima_student_265.pkl
Best ARIMA params: (1, 0, 0), MAE: 1.0443, RMSE: 1.2479

Training ARIMA for Student 266
ARIMA model for Student 266 saved to model/arima\arima_student_266.pkl
Best ARIMA params: (0, 1, 0), MAE: 1.2617, RMSE: 1.3505

Training ARIMA for Student 267
ARIMA model for Student 267 saved to model/arima\arima_student_267.pkl
Best ARIMA params: (0, 0, 2), MAE: 0.7408, RMSE: 0.8716

Training ARIMA for Student 268
ARIMA model for Student 268 saved to model/arima\arima_student_268.pkl
Best ARIMA params: (2, 0, 2), MAE: 1.0536, RMSE: 1.2542

Training ARIMA for Student 269
ARIMA model for Student 269 saved to model/ari

ARIMA model for Student 315 saved to model/arima\arima_student_315.pkl
Best ARIMA params: (0, 0, 0), MAE: 1.4731, RMSE: 1.6360

Training ARIMA for Student 316
ARIMA model for Student 316 saved to model/arima\arima_student_316.pkl
Best ARIMA params: (0, 1, 2), MAE: 0.8863, RMSE: 1.0348

Training ARIMA for Student 317
ARIMA model for Student 317 saved to model/arima\arima_student_317.pkl
Best ARIMA params: (2, 0, 0), MAE: 0.9230, RMSE: 1.1578

Training ARIMA for Student 318
ARIMA model for Student 318 saved to model/arima\arima_student_318.pkl
Best ARIMA params: (2, 0, 1), MAE: 1.2565, RMSE: 1.5172

Training ARIMA for Student 319
ARIMA model for Student 319 saved to model/arima\arima_student_319.pkl
Best ARIMA params: (1, 1, 2), MAE: 0.9670, RMSE: 1.2130

Training ARIMA for Student 320
ARIMA model for Student 320 saved to model/arima\arima_student_320.pkl
Best ARIMA params: (1, 2, 0), MAE: 1.4031, RMSE: 2.0630

Training ARIMA for Student 321
ARIMA model for Student 321 saved to model/ari

ARIMA model for Student 367 saved to model/arima\arima_student_367.pkl
Best ARIMA params: (2, 0, 2), MAE: 0.9978, RMSE: 1.1288

Training ARIMA for Student 368
ARIMA model for Student 368 saved to model/arima\arima_student_368.pkl
Best ARIMA params: (2, 1, 0), MAE: 0.5040, RMSE: 0.6030

Training ARIMA for Student 369
ARIMA model for Student 369 saved to model/arima\arima_student_369.pkl
Best ARIMA params: (1, 2, 1), MAE: 0.7358, RMSE: 0.9231

Training ARIMA for Student 370
ARIMA model for Student 370 saved to model/arima\arima_student_370.pkl
Best ARIMA params: (2, 0, 2), MAE: 0.8075, RMSE: 0.9460

Training ARIMA for Student 371
ARIMA model for Student 371 saved to model/arima\arima_student_371.pkl
Best ARIMA params: (1, 0, 1), MAE: 0.9299, RMSE: 1.0598

Training ARIMA for Student 372
ARIMA model for Student 372 saved to model/arima\arima_student_372.pkl
Best ARIMA params: (0, 2, 0), MAE: 0.6683, RMSE: 1.0006

Training ARIMA for Student 373
ARIMA model for Student 373 saved to model/ari

ARIMA model for Student 419 saved to model/arima\arima_student_419.pkl
Best ARIMA params: (0, 0, 2), MAE: 0.6949, RMSE: 0.8736

Training ARIMA for Student 420
ARIMA model for Student 420 saved to model/arima\arima_student_420.pkl
Best ARIMA params: (2, 0, 1), MAE: 0.6778, RMSE: 0.7512

Training ARIMA for Student 421
ARIMA model for Student 421 saved to model/arima\arima_student_421.pkl
Best ARIMA params: (0, 2, 2), MAE: 1.1514, RMSE: 1.3716

Training ARIMA for Student 422
ARIMA model for Student 422 saved to model/arima\arima_student_422.pkl
Best ARIMA params: (0, 0, 2), MAE: 0.9073, RMSE: 1.2093

Training ARIMA for Student 423
ARIMA model for Student 423 saved to model/arima\arima_student_423.pkl
Best ARIMA params: (2, 2, 1), MAE: 1.4550, RMSE: 1.7021

Training ARIMA for Student 424
ARIMA model for Student 424 saved to model/arima\arima_student_424.pkl
Best ARIMA params: (2, 2, 1), MAE: 0.7086, RMSE: 0.8810

Training ARIMA for Student 425
ARIMA model for Student 425 saved to model/ari

ARIMA model for Student 471 saved to model/arima\arima_student_471.pkl
Best ARIMA params: (1, 0, 0), MAE: 0.7697, RMSE: 0.9069

Training ARIMA for Student 472
ARIMA model for Student 472 saved to model/arima\arima_student_472.pkl
Best ARIMA params: (2, 0, 1), MAE: 1.0579, RMSE: 1.2190

Training ARIMA for Student 473
ARIMA model for Student 473 saved to model/arima\arima_student_473.pkl
Best ARIMA params: (1, 1, 2), MAE: 0.8974, RMSE: 1.0044

Training ARIMA for Student 474
ARIMA model for Student 474 saved to model/arima\arima_student_474.pkl
Best ARIMA params: (2, 0, 2), MAE: 1.0019, RMSE: 1.1543

Training ARIMA for Student 475
ARIMA model for Student 475 saved to model/arima\arima_student_475.pkl
Best ARIMA params: (2, 0, 0), MAE: 1.2197, RMSE: 1.4741

Training ARIMA for Student 476
ARIMA model for Student 476 saved to model/arima\arima_student_476.pkl
Best ARIMA params: (0, 0, 2), MAE: 1.0656, RMSE: 1.2608

Training ARIMA for Student 477
ARIMA model for Student 477 saved to model/ari

In [5]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.filterwarnings("ignore")

# Fungsi untuk memuat dan memproses data
def load_and_preprocess_data(file_path):
    data = pd.read_csv(file_path)
    data['Date'] = pd.to_datetime(data['Date'])
    data['Risk Level'] = data['Risk Level'].map({'Low': 0, 'Medium': 1, 'High': 2})
    data = data.dropna()
    
    numerical_features = ['Stress Level (GSR)', 'Sleep Hours', 'Anxiety Level', 'Mood Score']
    scaler = MinMaxScaler()
    data[numerical_features] = scaler.fit_transform(data[numerical_features])
    
    return data, scaler

# Fungsi untuk membagi data berdasarkan waktu
def split_data_by_time(student_data, train_days=24):
    student_data = student_data.sort_values('Date')
    unique_dates = student_data['Date'].unique()
    train_dates = unique_dates[:train_days]
    test_dates = unique_dates[train_days:]
    train_data = student_data[student_data['Date'].isin(train_dates)]
    test_data = student_data[student_data['Date'].isin(test_dates)]
    return train_data, test_data

# Fungsi untuk uji stasioneritas
def adf_test(series):
    result = adfuller(series.dropna())
    return result[1] < 0.05  # True jika stasioner

# Fungsi untuk melatih ARIMA dan menghasilkan prediksi untuk test set
def train_arima_for_lstm(student_data, student_id):
    train_data, test_data = split_data_by_time(student_data)
    
    best_mae = float('inf')
    best_model_fit = None
    
    if adf_test(train_data['Stress Level (GSR)']):
        for p in range(3):
            for d in range(2):
                for q in range(3):
                    try:
                        arima_model = ARIMA(train_data['Stress Level (GSR)'], order=(p, d, q))
                        arima_fit = arima_model.fit()
                        arima_pred = arima_fit.forecast(steps=len(test_data))
                        mae = mean_absolute_error(test_data['Stress Level (GSR)'], arima_pred)
                        if mae < best_mae:
                            best_mae = mae
                            best_model_fit = arima_fit
                    except:
                        continue
    
    arima_forecast = best_model_fit.forecast(steps=len(test_data)) if best_model_fit else np.zeros(len(test_data))
    return np.array(arima_forecast)  # Konversi ke NumPy array

# Model LSTM
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.dropout = nn.Dropout(0.4)
        self.risk_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim // 2, num_classes)
        )
        self.fc_stress = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        out, (h_n, _) = self.lstm(x)
        out = h_n[-1]
        out = self.bn(out)
        out = self.dropout(out)
        risk_logits = self.risk_head(out)
        stress = self.sigmoid(self.fc_stress(out))
        return risk_logits, stress.squeeze(1)

# Fungsi untuk melatih dan menyimpan model LSTM
def train_lstm(student_data, student_id, device, model_dir="model/lstm"):
    train_data, test_data = split_data_by_time(student_data)
    
    # Pastikan direktori model ada
    os.makedirs(model_dir, exist_ok=True)
    
    # Fitur dan target
    features = ['Stress Level (GSR)', 'Sleep Hours', 'Anxiety Level', 'Mood Score']
    target_cols = ['Risk Level', 'Stress Level (GSR)']
    
    # Persiapkan data
    X_train = train_data[features].values.reshape(-1, 1, len(features))
    X_test = test_data[features].values.reshape(-1, 1, len(features))
    y_train = train_data[target_cols].values
    
    # Dapatkan prediksi ARIMA untuk test set
    arima_forecast = train_arima_for_lstm(student_data, student_id)
    
    # Augmentasi data dengan prediksi ARIMA
    X_train_aug = np.concatenate((X_train, np.zeros((len(train_data), 1, 1))), axis=2)
    X_test_aug = np.concatenate((X_test, arima_forecast.reshape(-1, 1, 1)), axis=2)
    
    # Konversi ke tensor
    X_train_tensor = torch.tensor(X_train_aug, dtype=torch.float32).to(device)
    y_train_risk = torch.tensor(y_train[:, 0], dtype=torch.long).to(device)
    y_train_stress = torch.tensor(y_train[:, 1], dtype=torch.float32).to(device)
    
    # Inisialisasi model
    model = LSTMModel(input_dim=X_train_aug.shape[2], hidden_dim=64, num_classes=3).to(device)
    loss_risk = nn.CrossEntropyLoss()
    loss_stress = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    # Pelatihan
    for epoch in range(300):
        model.train()
        optimizer.zero_grad()
        risk_logits, stress_pred = model(X_train_tensor)
        loss1 = loss_risk(risk_logits, y_train_risk)
        loss2 = loss_stress(stress_pred, y_train_stress)
        loss = 1.5 * loss1 + 1.0 * loss2
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 50 == 0:
            print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")
    
    # Simpan model
    model_path = os.path.join(model_dir, f"lstm_student_{student_id}.pth")
    torch.save(model.state_dict(), model_path)
    print(f"LSTM model for Student {student_id} saved to {model_path}")
    
    return model

# Fungsi utama
def main():
    data, _ = load_and_preprocess_data('student_monnitoring_data.csv')
    user_groups = data.groupby('Student ID')
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Proses hanya 5 mahasiswa sebagai contoh
    for i, (student_id, student_data) in enumerate(user_groups):
        # if i >= 5:
        #     break
        print(f"\nTraining LSTM for Student {student_id}")
        train_lstm(student_data, student_id, device)


In [6]:
if __name__ == "__main__":
    main()


Training LSTM for Student 1
Epoch 50, Loss: 0.6189
Epoch 100, Loss: 0.4655
Epoch 150, Loss: 0.2854
Epoch 200, Loss: 0.5071
Epoch 250, Loss: 0.2427
Epoch 300, Loss: 0.1311
LSTM model for Student 1 saved to model/lstm\lstm_student_1.pth

Training LSTM for Student 2
Epoch 50, Loss: 0.6935
Epoch 100, Loss: 0.4845
Epoch 150, Loss: 0.2654
Epoch 200, Loss: 0.2345
Epoch 250, Loss: 0.2019
Epoch 300, Loss: 0.2388
LSTM model for Student 2 saved to model/lstm\lstm_student_2.pth

Training LSTM for Student 3
Epoch 50, Loss: 0.6070
Epoch 100, Loss: 0.3335
Epoch 150, Loss: 0.2524
Epoch 200, Loss: 0.1911
Epoch 250, Loss: 0.1353
Epoch 300, Loss: 0.0636
LSTM model for Student 3 saved to model/lstm\lstm_student_3.pth

Training LSTM for Student 4
Epoch 50, Loss: 0.9294
Epoch 100, Loss: 0.6679
Epoch 150, Loss: 0.4912
Epoch 200, Loss: 0.4347
Epoch 250, Loss: 0.4121
Epoch 300, Loss: 0.2643
LSTM model for Student 4 saved to model/lstm\lstm_student_4.pth

Training LSTM for Student 5
Epoch 50, Loss: 0.7407
Epoc

Epoch 250, Loss: 0.2126
Epoch 300, Loss: 0.1843
LSTM model for Student 35 saved to model/lstm\lstm_student_35.pth

Training LSTM for Student 36
Epoch 50, Loss: 0.7374
Epoch 100, Loss: 0.4366
Epoch 150, Loss: 0.3450
Epoch 200, Loss: 0.3826
Epoch 250, Loss: 0.3456
Epoch 300, Loss: 0.2236
LSTM model for Student 36 saved to model/lstm\lstm_student_36.pth

Training LSTM for Student 37
Epoch 50, Loss: 0.7715
Epoch 100, Loss: 0.5369
Epoch 150, Loss: 0.4812
Epoch 200, Loss: 0.2173
Epoch 250, Loss: 0.2230
Epoch 300, Loss: 0.1804
LSTM model for Student 37 saved to model/lstm\lstm_student_37.pth

Training LSTM for Student 38
Epoch 50, Loss: 0.9158
Epoch 100, Loss: 0.5534
Epoch 150, Loss: 0.4911
Epoch 200, Loss: 0.4894
Epoch 250, Loss: 0.5957
Epoch 300, Loss: 0.4208
LSTM model for Student 38 saved to model/lstm\lstm_student_38.pth

Training LSTM for Student 39
Epoch 50, Loss: 0.7450
Epoch 100, Loss: 0.5596
Epoch 150, Loss: 0.3812
Epoch 200, Loss: 0.4601
Epoch 250, Loss: 0.3458
Epoch 300, Loss: 0.3

Epoch 50, Loss: 0.8795
Epoch 100, Loss: 0.5952
Epoch 150, Loss: 0.4840
Epoch 200, Loss: 0.4711
Epoch 250, Loss: 0.4053
Epoch 300, Loss: 0.3092
LSTM model for Student 70 saved to model/lstm\lstm_student_70.pth

Training LSTM for Student 71
Epoch 50, Loss: 0.9510
Epoch 100, Loss: 0.8665
Epoch 150, Loss: 0.6858
Epoch 200, Loss: 0.5516
Epoch 250, Loss: 0.5591
Epoch 300, Loss: 0.3313
LSTM model for Student 71 saved to model/lstm\lstm_student_71.pth

Training LSTM for Student 72
Epoch 50, Loss: 0.9659
Epoch 100, Loss: 0.8095
Epoch 150, Loss: 0.8985
Epoch 200, Loss: 0.5776
Epoch 250, Loss: 0.5282
Epoch 300, Loss: 0.5039
LSTM model for Student 72 saved to model/lstm\lstm_student_72.pth

Training LSTM for Student 73
Epoch 50, Loss: 1.0422
Epoch 100, Loss: 0.7233
Epoch 150, Loss: 0.8457
Epoch 200, Loss: 0.6099
Epoch 250, Loss: 0.4685
Epoch 300, Loss: 0.3005
LSTM model for Student 73 saved to model/lstm\lstm_student_73.pth

Training LSTM for Student 74
Epoch 50, Loss: 0.7510
Epoch 100, Loss: 0.40

Epoch 200, Loss: 0.6536
Epoch 250, Loss: 0.4199
Epoch 300, Loss: 0.3917
LSTM model for Student 104 saved to model/lstm\lstm_student_104.pth

Training LSTM for Student 105
Epoch 50, Loss: 0.9241
Epoch 100, Loss: 0.7486
Epoch 150, Loss: 0.5552
Epoch 200, Loss: 0.5425
Epoch 250, Loss: 0.4693
Epoch 300, Loss: 0.3673
LSTM model for Student 105 saved to model/lstm\lstm_student_105.pth

Training LSTM for Student 106
Epoch 50, Loss: 0.9712
Epoch 100, Loss: 0.8845
Epoch 150, Loss: 0.6784
Epoch 200, Loss: 0.6736
Epoch 250, Loss: 0.4674
Epoch 300, Loss: 0.3710
LSTM model for Student 106 saved to model/lstm\lstm_student_106.pth

Training LSTM for Student 107
Epoch 50, Loss: 0.9734
Epoch 100, Loss: 0.6089
Epoch 150, Loss: 0.6337
Epoch 200, Loss: 0.4240
Epoch 250, Loss: 0.3419
Epoch 300, Loss: 0.3776
LSTM model for Student 107 saved to model/lstm\lstm_student_107.pth

Training LSTM for Student 108
Epoch 50, Loss: 1.1995
Epoch 100, Loss: 0.7582
Epoch 150, Loss: 0.5437
Epoch 200, Loss: 0.3811
Epoch 25

Epoch 150, Loss: 0.6568
Epoch 200, Loss: 0.4684
Epoch 250, Loss: 0.4665
Epoch 300, Loss: 0.3429
LSTM model for Student 138 saved to model/lstm\lstm_student_138.pth

Training LSTM for Student 139
Epoch 50, Loss: 0.9220
Epoch 100, Loss: 0.7377
Epoch 150, Loss: 0.6595
Epoch 200, Loss: 0.4990
Epoch 250, Loss: 0.3051
Epoch 300, Loss: 0.3234
LSTM model for Student 139 saved to model/lstm\lstm_student_139.pth

Training LSTM for Student 140
Epoch 50, Loss: 0.9737
Epoch 100, Loss: 0.5150
Epoch 150, Loss: 0.3263
Epoch 200, Loss: 0.2445
Epoch 250, Loss: 0.1359
Epoch 300, Loss: 0.0798
LSTM model for Student 140 saved to model/lstm\lstm_student_140.pth

Training LSTM for Student 141
Epoch 50, Loss: 0.7420
Epoch 100, Loss: 0.4565
Epoch 150, Loss: 0.3374
Epoch 200, Loss: 0.2152
Epoch 250, Loss: 0.1504
Epoch 300, Loss: 0.1082
LSTM model for Student 141 saved to model/lstm\lstm_student_141.pth

Training LSTM for Student 142
Epoch 50, Loss: 1.3081
Epoch 100, Loss: 0.8886
Epoch 150, Loss: 0.5776
Epoch 20

...

Epoch 50, Loss: 1.1700
Epoch 100, Loss: 0.9168
Epoch 150, Loss: 0.7820
Epoch 200, Loss: 0.7056
Epoch 250, Loss: 0.6603
Epoch 300, Loss: 0.5956
LSTM model for Student 342 saved to model/lstm\lstm_student_342.pth

Training LSTM for Student 343
Epoch 50, Loss: 0.6816
Epoch 100, Loss: 0.4082
Epoch 150, Loss: 0.3776
Epoch 200, Loss: 0.3452
Epoch 250, Loss: 0.3813
Epoch 300, Loss: 0.3691
LSTM model for Student 343 saved to model/lstm\lstm_student_343.pth

Training LSTM for Student 344
Epoch 50, Loss: 1.0554
Epoch 100, Loss: 0.7836
Epoch 150, Loss: 0.7551
Epoch 200, Loss: 0.4283
Epoch 250, Loss: 0.4562
Epoch 300, Loss: 0.4408
LSTM model for Student 344 saved to model/lstm\lstm_student_344.pth

Training LSTM for Student 345
Epoch 50, Loss: 0.7034
Epoch 100, Loss: 0.4817
Epoch 150, Loss: 0.3534
Epoch 200, Loss: 0.3455
Epoch 250, Loss: 0.3159
Epoch 300, Loss: 0.2842
LSTM model for Student 345 saved to model/lstm\lstm_student_345.pth

Training LSTM for Student 346
Epoch 50, Loss: 0.9802
Epoch 100

Epoch 50, Loss: 0.8910
Epoch 100, Loss: 0.6781
Epoch 150, Loss: 0.3338
Epoch 200, Loss: 0.3061
Epoch 250, Loss: 0.2441
Epoch 300, Loss: 0.1884
LSTM model for Student 376 saved to model/lstm\lstm_student_376.pth

Training LSTM for Student 377
Epoch 50, Loss: 0.9761
Epoch 100, Loss: 0.8042
Epoch 150, Loss: 0.5228
Epoch 200, Loss: 0.5815
Epoch 250, Loss: 0.4733
Epoch 300, Loss: 0.2246
LSTM model for Student 377 saved to model/lstm\lstm_student_377.pth

Training LSTM for Student 378
Epoch 50, Loss: 0.5769
Epoch 100, Loss: 0.2815
Epoch 150, Loss: 0.1006
Epoch 200, Loss: 0.1091
Epoch 250, Loss: 0.1210
Epoch 300, Loss: 0.0261
LSTM model for Student 378 saved to model/lstm\lstm_student_378.pth

Training LSTM for Student 379
Epoch 50, Loss: 0.9204
Epoch 100, Loss: 0.7777
Epoch 150, Loss: 0.4656
Epoch 200, Loss: 0.6165
Epoch 250, Loss: 0.4823
Epoch 300, Loss: 0.4582
LSTM model for Student 379 saved to model/lstm\lstm_student_379.pth

Training LSTM for Student 380
Epoch 50, Loss: 1.3620
Epoch 100

Epoch 50, Loss: 0.8008
Epoch 100, Loss: 0.6985
Epoch 150, Loss: 0.6052
Epoch 200, Loss: 0.4211
Epoch 250, Loss: 0.3804
Epoch 300, Loss: 0.3286
LSTM model for Student 410 saved to model/lstm\lstm_student_410.pth

Training LSTM for Student 411
Epoch 50, Loss: 0.7887
Epoch 100, Loss: 0.4263
Epoch 150, Loss: 0.3084
Epoch 200, Loss: 0.3036
Epoch 250, Loss: 0.2259
Epoch 300, Loss: 0.2518
LSTM model for Student 411 saved to model/lstm\lstm_student_411.pth

Training LSTM for Student 412
Epoch 50, Loss: 0.5757
Epoch 100, Loss: 0.2997
Epoch 150, Loss: 0.1789
Epoch 200, Loss: 0.1527
Epoch 250, Loss: 0.1444
Epoch 300, Loss: 0.0721
LSTM model for Student 412 saved to model/lstm\lstm_student_412.pth

Training LSTM for Student 413
Epoch 50, Loss: 0.5483
Epoch 100, Loss: 0.4649
Epoch 150, Loss: 0.3166
Epoch 200, Loss: 0.2950
Epoch 250, Loss: 0.3467
Epoch 300, Loss: 0.1926
LSTM model for Student 413 saved to model/lstm\lstm_student_413.pth

Training LSTM for Student 414
Epoch 50, Loss: 0.9870
Epoch 100

Epoch 50, Loss: 0.8929
Epoch 100, Loss: 0.5792
Epoch 150, Loss: 0.4728
Epoch 200, Loss: 0.3857
Epoch 250, Loss: 0.3708
Epoch 300, Loss: 0.2022
LSTM model for Student 444 saved to model/lstm\lstm_student_444.pth

Training LSTM for Student 445
Epoch 50, Loss: 1.0927
Epoch 100, Loss: 0.8584
Epoch 150, Loss: 0.7362
Epoch 200, Loss: 0.6447
Epoch 250, Loss: 0.4997
Epoch 300, Loss: 0.4920
LSTM model for Student 445 saved to model/lstm\lstm_student_445.pth

Training LSTM for Student 446
Epoch 50, Loss: 0.9512
Epoch 100, Loss: 0.6658
Epoch 150, Loss: 0.6270
Epoch 200, Loss: 0.4227
Epoch 250, Loss: 0.3570
Epoch 300, Loss: 0.2232
LSTM model for Student 446 saved to model/lstm\lstm_student_446.pth

Training LSTM for Student 447
Epoch 50, Loss: 0.6876
Epoch 100, Loss: 0.5640
Epoch 150, Loss: 0.4475
Epoch 200, Loss: 0.3874
Epoch 250, Loss: 0.3417
Epoch 300, Loss: 0.2450
LSTM model for Student 447 saved to model/lstm\lstm_student_447.pth

Training LSTM for Student 448
Epoch 50, Loss: 0.9548
Epoch 100

Epoch 50, Loss: 0.6765
Epoch 100, Loss: 0.6093
Epoch 150, Loss: 0.4457
Epoch 200, Loss: 0.3274
Epoch 250, Loss: 0.3172
Epoch 300, Loss: 0.2167
LSTM model for Student 478 saved to model/lstm\lstm_student_478.pth

Training LSTM for Student 479
Epoch 50, Loss: 0.5537
Epoch 100, Loss: 0.2679
Epoch 150, Loss: 0.2505
Epoch 200, Loss: 0.1279
Epoch 250, Loss: 0.1235
Epoch 300, Loss: 0.1013
LSTM model for Student 479 saved to model/lstm\lstm_student_479.pth

Training LSTM for Student 480
Epoch 50, Loss: 1.0617
Epoch 100, Loss: 0.7899
Epoch 150, Loss: 0.6578
Epoch 200, Loss: 0.5078
Epoch 250, Loss: 0.4452
Epoch 300, Loss: 0.4405
LSTM model for Student 480 saved to model/lstm\lstm_student_480.pth

Training LSTM for Student 481
Epoch 50, Loss: 1.0056
Epoch 100, Loss: 0.7501
Epoch 150, Loss: 0.6089
Epoch 200, Loss: 0.5064
Epoch 250, Loss: 0.5623
Epoch 300, Loss: 0.4475
LSTM model for Student 481 saved to model/lstm\lstm_student_481.pth

Training LSTM for Student 482
Epoch 50, Loss: 0.7197
Epoch 100

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings("ignore")

# Fungsi untuk menghitung MAPE
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero = y_true != 0
    return np.mean(np.abs((y_true[non_zero] - y_pred[non_zero]) / y_true[non_zero])) * 100

# Fungsi untuk memuat dan memproses data
def load_and_preprocess_data(file_path):
    data = pd.read_csv(file_path)
    data['Date'] = pd.to_datetime(data['Date'])
    data['Risk Level'] = data['Risk Level'].map({'Low': 0, 'Medium': 1, 'High': 2})
    data = data.dropna()
    
    numerical_features = ['Stress Level (GSR)', 'Sleep Hours', 'Anxiety Level', 'Mood Score']
    scaler = MinMaxScaler()
    data[numerical_features] = scaler.fit_transform(data[numerical_features])
    
    return data, scaler

# Fungsi untuk membagi data berdasarkan waktu
def split_data_by_time(student_data, train_days=24):
    student_data = student_data.sort_values('Date')
    unique_dates = student_data['Date'].unique()
    train_dates = unique_dates[:train_days]
    test_dates = unique_dates[train_days:]
    train_data = student_data[student_data['Date'].isin(train_dates)]
    test_data = student_data[student_data['Date'].isin(test_dates)]
    return train_data, test_data

# Model LSTM
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.dropout = nn.Dropout(0.4)
        self.risk_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim // 2, num_classes)
        )
        self.fc_stress = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        out, (h_n, _) = self.lstm(x)
        out = h_n[-1]
        out = self.bn(out)
        out = self.dropout(out)
        risk_logits = self.risk_head(out)
        stress = self.sigmoid(self.fc_stress(out))
        return risk_logits, stress.squeeze(1)

# Fungsi untuk memuat model ARIMA
def load_arima_model(student_id, model_dir="model/arima"):
    model_path = os.path.join(model_dir, f"arima_student_{student_id}.pkl")
    if os.path.exists(model_path):
        with open(model_path, 'rb') as f:
            return pickle.load(f)
    raise FileNotFoundError(f"ARIMA model for Student {student_id} not found at {model_path}")

# Fungsi untuk memuat model LSTM
def load_lstm_model(student_id, input_dim, model_dir="model/lstm"):
    model_path = os.path.join(model_dir, f"lstm_student_{student_id}.pth")
    if os.path.exists(model_path):
        model = LSTMModel(input_dim=input_dim, hidden_dim=64, num_classes=3)
        model.load_state_dict(torch.load(model_path))
        return model
    raise FileNotFoundError(f"LSTM model for Student {student_id} not found at {model_path}")

# Fungsi untuk prediksi hybrid per mahasiswa
def predict_hybrid(student_data, student_id, scaler, device):
    train_data, test_data = split_data_by_time(student_data)
    
    # Fitur dan target
    features = ['Stress Level (GSR)', 'Sleep Hours', 'Anxiety Level', 'Mood Score']
    target_cols = ['Risk Level', 'Stress Level (GSR)']
    
    # Persiapkan data
    X_train = train_data[features].values.reshape(-1, 1, len(features))
    X_test = test_data[features].values.reshape(-1, 1, len(features))
    y_train = train_data[target_cols].values
    y_test = test_data[target_cols].values
    
    # Muat model ARIMA
    arima_model = load_arima_model(student_id)
    arima_forecast = arima_model.forecast(steps=len(test_data))
    
    # Augmentasi data dengan prediksi ARIMA
    X_train_aug = np.concatenate((X_train, np.zeros((len(train_data), 1, 1))), axis=2)
    X_test_aug = np.concatenate((X_test, arima_forecast.values.reshape(-1, 1, 1)), axis=2)
    
    # Konversi ke tensor
    X_train_tensor = torch.tensor(X_train_aug, dtype=torch.float32).to(device)
    X_test_tensor = torch.tensor(X_test_aug, dtype=torch.float32).to(device)
    y_train_risk = torch.tensor(y_train[:, 0], dtype=torch.long).to(device)
    y_train_stress = torch.tensor(y_train[:, 1], dtype=torch.float32).to(device)
    y_test_risk = y_test[:, 0].astype(int)
    y_test_stress = y_test[:, 1]
    
    # Muat model LSTM
    model = load_lstm_model(student_id, input_dim=X_train_aug.shape[2])
    model.to(device)
    model.eval()
    
    # Evaluasi pada test set
    with torch.no_grad():
        test_logits, test_stress = model(X_test_tensor)
        risk_pred_test = torch.argmax(test_logits, dim=1).cpu().numpy()
        stress_pred_test = test_stress.cpu().numpy()
    
    # Metrik evaluasi
    mae_stress = mean_absolute_error(y_test_stress, stress_pred_test)
    rmse_stress = np.sqrt(mean_squared_error(y_test_stress, stress_pred_test))
    mape_stress = mean_absolute_percentage_error(y_test_stress, stress_pred_test)
    mae_risk = mean_absolute_error(y_test_risk, risk_pred_test)
    acc_risk = np.mean(risk_pred_test == y_test_risk)
    
    print(f"\n[Student {student_id}] Evaluation:")
    print(f"MAE Stress: {mae_stress:.4f}, RMSE Stress: {rmse_stress:.4f}, MAPE Stress: {mape_stress:.2f}%")
    print(f"MAE Risk: {mae_risk:.4f}, Accuracy Risk: {acc_risk * 100:.2f}%")
    
    # Prediksi 10 hari ke depan
    n_steps = 5
    window_input = X_test_aug[-n_steps:].copy()
    window_input = torch.tensor(window_input, dtype=torch.float32).to(device)
    future_preds = []
    
    arima_future = arima_model.forecast(steps=10).values
    arima_future = np.clip(arima_future, 0, 1)
    lstm_stress_series = []
    
    with torch.no_grad():
        for day in range(10):
            risk_logits_next, stress_pred_next = model(window_input)
            lstm_stress = stress_pred_next[-1].item()
            lstm_stress_series.append(lstm_stress)
            
            smoothed_stress = np.mean(lstm_stress_series[-3:]) if len(lstm_stress_series) >= 3 else lstm_stress
            alpha = np.clip(0.8 - (day * 0.05), 0.3, 0.8)
            combined_stress = alpha * smoothed_stress + (1 - alpha) * arima_future[day]
            combined_stress = np.clip(combined_stress + np.random.normal(0, 0.02), 0, 1)
            
            risk_level_pred = torch.argmax(risk_logits_next[-1]).item()
            last_input = window_input[-1, 0].cpu().numpy()
            next_features = np.clip(last_input[1:-1] + (np.random.rand(3) - 0.5) * 0.02, 0, 1)
            new_input = np.concatenate([[combined_stress], next_features, [arima_future[day]]])
            new_input_tensor = torch.tensor(new_input, dtype=torch.float32).reshape(1, 1, -1).to(device)
            window_input = torch.cat((window_input[1:], new_input_tensor), dim=0)
            future_preds.append([risk_level_pred, combined_stress])
    
    future_preds = np.array(future_preds)
    
    # Visualisasi
    plt.figure(figsize=(12, 10))
    future_dates = pd.date_range(start=test_data['Date'].iloc[-1] + pd.Timedelta(days=1), periods=10)
    full_data = pd.concat([train_data, test_data])
    
    plt.subplot(2, 1, 1)
    plt.plot(full_data['Date'], full_data['Stress Level (GSR)'], label='Actual Stress', color='blue')
    plt.plot(test_data['Date'], stress_pred_test, label='Predicted Stress (Test)', color='red', linestyle='--')
    plt.plot(future_dates, future_preds[:, 1], label='Forecast Stress', color='purple', linestyle=':')
    plt.title(f"Stress Level - Student {student_id}")
    plt.xlabel('Date')
    plt.ylabel('Stress Level (GSR)')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(2, 1, 2)
    plt.plot(full_data['Date'], full_data['Risk Level'], label='Actual Risk', color='green')
    plt.plot(test_data['Date'], risk_pred_test, label='Predicted Risk (Test)', color='orange', linestyle='--')
    plt.plot(future_dates, future_preds[:, 0], label='Forecast Risk', color='brown', linestyle=':')
    plt.title(f"Risk Level - Student {student_id}")
    plt.xlabel('Date')
    plt.ylabel('Risk Level')
    plt.yticks([0, 1, 2], ['Low', 'Medium', 'High'])
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(f"student_{student_id}_forecast.png")
    plt.close()

# Fungsi utama
def main():
    data, scaler = load_and_preprocess_data('student_monnitoring_data.csv')
    user_groups = data.groupby('Student ID')
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Proses hanya 5 mahasiswa sebagai contoh
    for i, (student_id, student_data) in enumerate(user_groups):
        # if i >= 5:
        #     break
        print(f"\nProcessing Hybrid for Student {student_id}")
        predict_hybrid(student_data, student_id, scaler, device)


In [8]:
if __name__ == "__main__":
    main()


Processing Hybrid for Student 1

[Student 1] Evaluation:
MAE Stress: 0.2471, RMSE Stress: 0.2776, MAPE Stress: 49.03%
MAE Risk: 0.6667, Accuracy Risk: 50.00%

Processing Hybrid for Student 2

[Student 2] Evaluation:
MAE Stress: 0.0815, RMSE Stress: 0.0849, MAPE Stress: 44.46%
MAE Risk: 0.8333, Accuracy Risk: 50.00%

Processing Hybrid for Student 3

[Student 3] Evaluation:
MAE Stress: 0.1019, RMSE Stress: 0.1142, MAPE Stress: 41.99%
MAE Risk: 1.0000, Accuracy Risk: 33.33%

Processing Hybrid for Student 4

[Student 4] Evaluation:
MAE Stress: 0.1183, RMSE Stress: 0.1446, MAPE Stress: 63.78%
MAE Risk: 0.1667, Accuracy Risk: 83.33%

Processing Hybrid for Student 5

[Student 5] Evaluation:
MAE Stress: 0.7113, RMSE Stress: 0.7368, MAPE Stress: 7395.61%
MAE Risk: 1.0000, Accuracy Risk: 33.33%

Processing Hybrid for Student 6

[Student 6] Evaluation:
MAE Stress: 0.4496, RMSE Stress: 0.5046, MAPE Stress: 81.49%
MAE Risk: 0.5000, Accuracy Risk: 66.67%

Processing Hybrid for Student 7

[Student 7


Processing Hybrid for Student 52

[Student 52] Evaluation:
MAE Stress: 0.4529, RMSE Stress: 0.5031, MAPE Stress: 95.93%
MAE Risk: 1.5000, Accuracy Risk: 16.67%

Processing Hybrid for Student 53

[Student 53] Evaluation:
MAE Stress: 0.1466, RMSE Stress: 0.1580, MAPE Stress: 46.72%
MAE Risk: 0.6667, Accuracy Risk: 50.00%

Processing Hybrid for Student 54

[Student 54] Evaluation:
MAE Stress: 0.5308, RMSE Stress: 0.5315, MAPE Stress: 68.71%
MAE Risk: 0.1667, Accuracy Risk: 83.33%

Processing Hybrid for Student 55

[Student 55] Evaluation:
MAE Stress: 0.0464, RMSE Stress: 0.0570, MAPE Stress: 8.38%
MAE Risk: 0.5000, Accuracy Risk: 66.67%

Processing Hybrid for Student 56

[Student 56] Evaluation:
MAE Stress: 0.0909, RMSE Stress: 0.0944, MAPE Stress: 96.06%
MAE Risk: 1.0000, Accuracy Risk: 50.00%

Processing Hybrid for Student 57

[Student 57] Evaluation:
MAE Stress: 0.3640, RMSE Stress: 0.4016, MAPE Stress: 76.59%
MAE Risk: 1.3333, Accuracy Risk: 16.67%

Processing Hybrid for Student 58




Processing Hybrid for Student 103

[Student 103] Evaluation:
MAE Stress: 0.2499, RMSE Stress: 0.2617, MAPE Stress: 439.00%
MAE Risk: 0.5000, Accuracy Risk: 66.67%

Processing Hybrid for Student 104

[Student 104] Evaluation:
MAE Stress: 0.3203, RMSE Stress: 0.3246, MAPE Stress: 85.97%
MAE Risk: 0.6667, Accuracy Risk: 50.00%

Processing Hybrid for Student 105

[Student 105] Evaluation:
MAE Stress: 0.1112, RMSE Stress: 0.1209, MAPE Stress: 33.40%
MAE Risk: 0.1667, Accuracy Risk: 83.33%

Processing Hybrid for Student 106

[Student 106] Evaluation:
MAE Stress: 0.6764, RMSE Stress: 0.6784, MAPE Stress: 79.38%
MAE Risk: 0.5000, Accuracy Risk: 66.67%

Processing Hybrid for Student 107

[Student 107] Evaluation:
MAE Stress: 0.2626, RMSE Stress: 0.2937, MAPE Stress: 71.08%
MAE Risk: 0.8333, Accuracy Risk: 50.00%

Processing Hybrid for Student 108

[Student 108] Evaluation:
MAE Stress: 0.5688, RMSE Stress: 0.6267, MAPE Stress: 459.22%
MAE Risk: 1.3333, Accuracy Risk: 16.67%

Processing Hybrid f


Processing Hybrid for Student 154

[Student 154] Evaluation:
MAE Stress: 0.5277, RMSE Stress: 0.5651, MAPE Stress: 98.39%
MAE Risk: 0.8333, Accuracy Risk: 33.33%

Processing Hybrid for Student 155

[Student 155] Evaluation:
MAE Stress: 0.1042, RMSE Stress: 0.1285, MAPE Stress: 15.74%
MAE Risk: 0.6667, Accuracy Risk: 66.67%

Processing Hybrid for Student 156

[Student 156] Evaluation:
MAE Stress: 0.2573, RMSE Stress: 0.2960, MAPE Stress: 65.57%
MAE Risk: 0.1667, Accuracy Risk: 83.33%

Processing Hybrid for Student 157

[Student 157] Evaluation:
MAE Stress: 0.2767, RMSE Stress: 0.2809, MAPE Stress: 88.28%
MAE Risk: 1.1667, Accuracy Risk: 33.33%

Processing Hybrid for Student 158

[Student 158] Evaluation:
MAE Stress: 0.5319, RMSE Stress: 0.5935, MAPE Stress: 317.27%
MAE Risk: 0.3333, Accuracy Risk: 66.67%

Processing Hybrid for Student 159

[Student 159] Evaluation:
MAE Stress: 0.2703, RMSE Stress: 0.3130, MAPE Stress: 66.39%
MAE Risk: 0.8333, Accuracy Risk: 50.00%

Processing Hybrid fo


Processing Hybrid for Student 205

[Student 205] Evaluation:
MAE Stress: 0.3733, RMSE Stress: 0.3784, MAPE Stress: 67.57%
MAE Risk: 0.0000, Accuracy Risk: 100.00%

Processing Hybrid for Student 206

[Student 206] Evaluation:
MAE Stress: 0.1584, RMSE Stress: 0.1594, MAPE Stress: 45.20%
MAE Risk: 0.6667, Accuracy Risk: 50.00%

Processing Hybrid for Student 207

[Student 207] Evaluation:
MAE Stress: 0.2636, RMSE Stress: 0.3161, MAPE Stress: 91.41%
MAE Risk: 1.0000, Accuracy Risk: 16.67%

Processing Hybrid for Student 208

[Student 208] Evaluation:
MAE Stress: 0.4672, RMSE Stress: 0.5743, MAPE Stress: 272.78%
MAE Risk: 1.1667, Accuracy Risk: 33.33%

Processing Hybrid for Student 209

[Student 209] Evaluation:
MAE Stress: 0.1298, RMSE Stress: 0.1440, MAPE Stress: 48.21%
MAE Risk: 0.8333, Accuracy Risk: 50.00%

Processing Hybrid for Student 210

[Student 210] Evaluation:
MAE Stress: 0.0666, RMSE Stress: 0.0817, MAPE Stress: 56.14%
MAE Risk: 0.1667, Accuracy Risk: 83.33%

Processing Hybrid f


Processing Hybrid for Student 256

[Student 256] Evaluation:
MAE Stress: 0.1506, RMSE Stress: 0.1546, MAPE Stress: 320.80%
MAE Risk: 1.0000, Accuracy Risk: 33.33%

Processing Hybrid for Student 257

[Student 257] Evaluation:
MAE Stress: 0.1721, RMSE Stress: 0.1903, MAPE Stress: 61.68%
MAE Risk: 0.3333, Accuracy Risk: 66.67%

Processing Hybrid for Student 258

[Student 258] Evaluation:
MAE Stress: 0.3295, RMSE Stress: 0.4036, MAPE Stress: 83.20%
MAE Risk: 0.8333, Accuracy Risk: 33.33%

Processing Hybrid for Student 259

[Student 259] Evaluation:
MAE Stress: 0.2052, RMSE Stress: 0.2356, MAPE Stress: 145.00%
MAE Risk: 0.8333, Accuracy Risk: 50.00%

Processing Hybrid for Student 260

[Student 260] Evaluation:
MAE Stress: 0.2859, RMSE Stress: 0.3330, MAPE Stress: 76.45%
MAE Risk: 0.6667, Accuracy Risk: 50.00%

Processing Hybrid for Student 261

[Student 261] Evaluation:
MAE Stress: 0.0574, RMSE Stress: 0.0632, MAPE Stress: 44.10%
MAE Risk: 0.1667, Accuracy Risk: 83.33%

Processing Hybrid f


Processing Hybrid for Student 307

[Student 307] Evaluation:
MAE Stress: 0.2837, RMSE Stress: 0.2998, MAPE Stress: 87.90%
MAE Risk: 0.5000, Accuracy Risk: 66.67%

Processing Hybrid for Student 308

[Student 308] Evaluation:
MAE Stress: 0.2650, RMSE Stress: 0.2891, MAPE Stress: 105.54%
MAE Risk: 1.0000, Accuracy Risk: 33.33%

Processing Hybrid for Student 309

[Student 309] Evaluation:
MAE Stress: 0.3260, RMSE Stress: 0.3365, MAPE Stress: 59.01%
MAE Risk: 0.1667, Accuracy Risk: 83.33%

Processing Hybrid for Student 310

[Student 310] Evaluation:
MAE Stress: 0.1795, RMSE Stress: 0.2087, MAPE Stress: 35.79%
MAE Risk: 0.1667, Accuracy Risk: 83.33%

Processing Hybrid for Student 311

[Student 311] Evaluation:
MAE Stress: 0.0589, RMSE Stress: 0.0739, MAPE Stress: 10.69%
MAE Risk: 0.5000, Accuracy Risk: 66.67%

Processing Hybrid for Student 312

[Student 312] Evaluation:
MAE Stress: 0.0492, RMSE Stress: 0.0565, MAPE Stress: 15.71%
MAE Risk: 0.0000, Accuracy Risk: 100.00%

Processing Hybrid f


Processing Hybrid for Student 358

[Student 358] Evaluation:
MAE Stress: 0.1040, RMSE Stress: 0.1306, MAPE Stress: 47.95%
MAE Risk: 1.1667, Accuracy Risk: 16.67%

Processing Hybrid for Student 359

[Student 359] Evaluation:
MAE Stress: 0.1910, RMSE Stress: 0.2124, MAPE Stress: 359.62%
MAE Risk: 0.6667, Accuracy Risk: 50.00%

Processing Hybrid for Student 360

[Student 360] Evaluation:
MAE Stress: 0.0359, RMSE Stress: 0.0412, MAPE Stress: 24.20%
MAE Risk: 0.6667, Accuracy Risk: 33.33%

Processing Hybrid for Student 361

[Student 361] Evaluation:
MAE Stress: 0.0364, RMSE Stress: 0.0428, MAPE Stress: 12.37%
MAE Risk: 1.0000, Accuracy Risk: 0.00%

Processing Hybrid for Student 362

[Student 362] Evaluation:
MAE Stress: 0.1440, RMSE Stress: 0.1448, MAPE Stress: 35.16%
MAE Risk: 0.8333, Accuracy Risk: 33.33%

Processing Hybrid for Student 363

[Student 363] Evaluation:
MAE Stress: 0.3735, RMSE Stress: 0.4218, MAPE Stress: 86.01%
MAE Risk: 0.5000, Accuracy Risk: 66.67%

Processing Hybrid for


Processing Hybrid for Student 409

[Student 409] Evaluation:
MAE Stress: 0.4080, RMSE Stress: 0.4956, MAPE Stress: 145.83%
MAE Risk: 0.6667, Accuracy Risk: 66.67%

Processing Hybrid for Student 410

[Student 410] Evaluation:
MAE Stress: 0.0914, RMSE Stress: 0.1031, MAPE Stress: 37.49%
MAE Risk: 0.3333, Accuracy Risk: 66.67%

Processing Hybrid for Student 411

[Student 411] Evaluation:
MAE Stress: 0.4264, RMSE Stress: 0.4872, MAPE Stress: 122.22%
MAE Risk: 0.6667, Accuracy Risk: 66.67%

Processing Hybrid for Student 412

[Student 412] Evaluation:
MAE Stress: 0.2721, RMSE Stress: 0.3195, MAPE Stress: 76.20%
MAE Risk: 0.1667, Accuracy Risk: 83.33%

Processing Hybrid for Student 413

[Student 413] Evaluation:
MAE Stress: 0.1339, RMSE Stress: 0.1385, MAPE Stress: 34.42%
MAE Risk: 0.6667, Accuracy Risk: 50.00%

Processing Hybrid for Student 414

[Student 414] Evaluation:
MAE Stress: 0.1790, RMSE Stress: 0.1876, MAPE Stress: 38.42%
MAE Risk: 0.3333, Accuracy Risk: 83.33%

Processing Hybrid f


Processing Hybrid for Student 460

[Student 460] Evaluation:
MAE Stress: 0.3351, RMSE Stress: 0.3532, MAPE Stress: 69.98%
MAE Risk: 0.5000, Accuracy Risk: 66.67%

Processing Hybrid for Student 461

[Student 461] Evaluation:
MAE Stress: 0.2328, RMSE Stress: 0.2666, MAPE Stress: 53.36%
MAE Risk: 0.6667, Accuracy Risk: 66.67%

Processing Hybrid for Student 462

[Student 462] Evaluation:
MAE Stress: 0.3147, RMSE Stress: 0.3974, MAPE Stress: 122.58%
MAE Risk: 0.3333, Accuracy Risk: 83.33%

Processing Hybrid for Student 463

[Student 463] Evaluation:
MAE Stress: 0.0975, RMSE Stress: 0.1051, MAPE Stress: 108.68%
MAE Risk: 1.1667, Accuracy Risk: 33.33%

Processing Hybrid for Student 464

[Student 464] Evaluation:
MAE Stress: 0.5517, RMSE Stress: 0.5548, MAPE Stress: 82.31%
MAE Risk: 0.3333, Accuracy Risk: 66.67%

Processing Hybrid for Student 465

[Student 465] Evaluation:
MAE Stress: 0.0789, RMSE Stress: 0.0860, MAPE Stress: 17.92%
MAE Risk: 0.5000, Accuracy Risk: 66.67%

Processing Hybrid f

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings("ignore")

# Fungsi untuk menghitung MAPE
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero = y_true != 0
    return np.mean(np.abs((y_true[non_zero] - y_pred[non_zero]) / y_true[non_zero])) * 100

# Fungsi untuk memuat dan memproses data
def load_and_preprocess_data(file_path):
    data = pd.read_csv(file_path)
    data['Date'] = pd.to_datetime(data['Date'])
    data['Risk Level'] = data['Risk Level'].map({'Low': 0, 'Medium': 1, 'High': 2})
    data = data.dropna()
    
    numerical_features = ['Stress Level (GSR)', 'Sleep Hours', 'Anxiety Level', 'Mood Score']
    scaler = MinMaxScaler()
    data[numerical_features] = scaler.fit_transform(data[numerical_features])
    
    return data, scaler

# Fungsi untuk membagi data berdasarkan waktu
def split_data_by_time(student_data, train_days=24):
    student_data = student_data.sort_values('Date')
    unique_dates = student_data['Date'].unique()
    train_dates = unique_dates[:train_days]
    test_dates = unique_dates[train_days:]
    train_data = student_data[student_data['Date'].isin(train_dates)]
    test_data = student_data[student_data['Date'].isin(test_dates)]
    return train_data, test_data

# Model LSTM
class LSTMModel(nn.Module):
    def _init_(self, input_dim, hidden_dim, num_classes):
        super(LSTMModel, self)._init_()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.dropout = nn.Dropout(0.4)
        self.risk_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim // 2, num_classes)
        )
        self.fc_stress = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        out, (h_n, _) = self.lstm(x)
        out = h_n[-1]
        out = self.bn(out)
        out = self.dropout(out)
        risk_logits = self.risk_head(out)
        stress = self.sigmoid(self.fc_stress(out))
        return risk_logits, stress.squeeze(1)

# Fungsi untuk memuat model ARIMA
def load_arima_model(student_id, model_dir="model/arima"):
    model_path = os.path.join(model_dir, f"arima_student_{student_id}.pkl")
    if os.path.exists(model_path):
        with open(model_path, 'rb') as f:
            return pickle.load(f)
    raise FileNotFoundError(f"ARIMA model for Student {student_id} not found at {model_path}")

# Fungsi untuk memuat model LSTM
def load_lstm_model(student_id, input_dim, model_dir="model/lstm"):
    model_path = os.path.join(model_dir, f"lstm_student_{student_id}.pth")
    if os.path.exists(model_path):
        model = LSTMModel(input_dim=input_dim, hidden_dim=64, num_classes=3)
        model.load_state_dict(torch.load(model_path))
        return model
    raise FileNotFoundError(f"LSTM model for Student {student_id} not found at {model_path}")

# Fungsi untuk prediksi hybrid per mahasiswa
def predict_hybrid(student_data, student_id, scaler, device):
    train_data, test_data = split_data_by_time(student_data)
    
    # Fitur dan target
    features = ['Stress Level (GSR)', 'Sleep Hours', 'Anxiety Level', 'Mood Score']
    target_cols = ['Risk Level', 'Stress Level (GSR)']
    
    # Persiapkan data
    X_train = train_data[features].values.reshape(-1, 1, len(features))
    X_test = test_data[features].values.reshape(-1, 1, len(features))
    y_train = train_data[target_cols].values
    y_test = test_data[target_cols].values
    
    # Muat model ARIMA
    arima_model = load_arima_model(student_id)
    arima_forecast = arima_model.forecast(steps=len(test_data))
    
    # Augmentasi data dengan prediksi ARIMA
    X_train_aug = np.concatenate((X_train, np.zeros((len(train_data), 1, 1))), axis=2)
    X_test_aug = np.concatenate((X_test, arima_forecast.values.reshape(-1, 1, 1)), axis=2)
    
    # Konversi ke tensor
    X_train_tensor = torch.tensor(X_train_aug, dtype=torch.float32).to(device)
    X_test_tensor = torch.tensor(X_test_aug, dtype=torch.float32).to(device)
    y_train_risk = torch.tensor(y_train[:, 0], dtype=torch.long).to(device)
    y_train_stress = torch.tensor(y_train[:, 1], dtype=torch.float32).to(device)
    y_test_risk = y_test[:, 0].astype(int)
    y_test_stress = y_test[:, 1]
    
    # Muat model LSTM
    model = load_lstm_model(student_id, input_dim=X_train_aug.shape[2])
    model.to(device)
    model.eval()
    
    # Evaluasi pada test set
    with torch.no_grad():
        test_logits, test_stress = model(X_test_tensor)
        risk_pred_test = torch.argmax(test_logits, dim=1).cpu().numpy()
        stress_pred_test = test_stress.cpu().numpy()
    
    # Metrik evaluasi
    mae_stress = mean_absolute_error(y_test_stress, stress_pred_test)
    rmse_stress = np.sqrt(mean_squared_error(y_test_stress, stress_pred_test))
    mape_stress = mean_absolute_percentage_error(y_test_stress, stress_pred_test)
    mae_risk = mean_absolute_error(y_test_risk, risk_pred_test)
    acc_risk = np.mean(risk_pred_test == y_test_risk)
    
    print(f"\n[Student {student_id}] Evaluation:")
    print(f"MAE Stress: {mae_stress:.4f}, RMSE Stress: {rmse_stress:.4f}, MAPE Stress: {mape_stress:.2f}%")
    print(f"MAE Risk: {mae_risk:.4f}, Accuracy Risk: {acc_risk * 100:.2f}%")
    
    # Prediksi 10 hari ke depan
    n_steps = 5
    window_input = X_test_aug[-n_steps:].copy()
    window_input = torch.tensor(window_input, dtype=torch.float32).to(device)
    future_preds = []
    
    arima_future = arima_model.forecast(steps=10).values
    arima_future = np.clip(arima_future, 0, 1)
    lstm_stress_series = []
    
    with torch.no_grad():
        for day in range(10):
            risk_logits_next, stress_pred_next = model(window_input)
            lstm_stress = stress_pred_next[-1].item()
            lstm_stress_series.append(lstm_stress)
            
            smoothed_stress = np.mean(lstm_stress_series[-3:]) if len(lstm_stress_series) >= 3 else lstm_stress
            alpha = np.clip(0.8 - (day * 0.05), 0.3, 0.8)
            combined_stress = alpha * smoothed_stress + (1 - alpha) * arima_future[day]
            combined_stress = np.clip(combined_stress + np.random.normal(0, 0.02), 0, 1)
            
            risk_level_pred = torch.argmax(risk_logits_next[-1]).item()
            last_input = window_input[-1, 0].cpu().numpy()
            next_features = np.clip(last_input[1:-1] + (np.random.rand(3) - 0.5) * 0.02, 0, 1)
            new_input = np.concatenate([[combined_stress], next_features, [arima_future[day]]])
            new_input_tensor = torch.tensor(new_input, dtype=torch.float32).reshape(1, 1, -1).to(device)
            window_input = torch.cat((window_input[1:], new_input_tensor), dim=0)
            future_preds.append([risk_level_pred, combined_stress])
    
    future_preds = np.array(future_preds)
    
    # Persiapan data visualisasi
    future_dates = pd.date_range(start=test_data['Date'].iloc[-1] + pd.Timedelta(days=1), periods=10)
    full_data = pd.concat([train_data, test_data])
    
    plt.figure(figsize=(12, 10))
    
    # -------- Subplot 1: Stress Level --------
    plt.subplot(2, 1, 1)
    plt.plot(full_data['Date'], full_data['Stress Level (GSR)'], label='Actual Stress', color='blue')
    plt.plot(test_data['Date'], stress_pred_test, label='Predicted Stress (Test)', color='red', linestyle='--')
    
    # Garis penyambung dari prediksi terakhir ke forecast pertama (Stress)
    plt.plot(
        [test_data['Date'].iloc[-1], future_dates[0]],
        [stress_pred_test[-1], future_preds[0, 1]],
        color='gray', linestyle='--', alpha=0.6, label='nolegend'
    )
    
    plt.plot(future_dates, future_preds[:, 1], label='Forecast Stress (Next 10 Days)', color='purple', linestyle=':')
    
    plt.title(f"Stress Level Forecast for Student {student_id}")
    plt.xlabel('Date')
    plt.ylabel('Stress Level (GSR)')
    plt.legend()
    plt.grid(True)
    
    # -------- Subplot 2: Risk Level --------
    plt.subplot(2, 1, 2)
    plt.plot(full_data['Date'], full_data['Risk Level'], label='Actual Risk Level', color='green')
    plt.plot(test_data['Date'], risk_pred_test, label='Predicted Risk Level (Test)', color='orange', linestyle='--')
    
    # Garis penyambung dari prediksi terakhir ke forecast pertama (Risk)
    plt.plot(
        [test_data['Date'].iloc[-1], future_dates[0]],
        [risk_pred_test[-1], future_preds[0, 0]],
        color='gray', linestyle='--', alpha=0.6, label='nolegend'
    )
    
    plt.plot(future_dates, future_preds[:, 0], label='Forecast Risk Level (Next 10 Days)', color='brown', linestyle=':')
    
    plt.title(f"Risk Level Forecast for Student {student_id}")
    plt.xlabel('Date')
    plt.ylabel('Risk Level')
    plt.yticks([0, 1, 2], ['Low', 'Medium', 'High'])
    plt.legend()
    plt.grid(True)
    
    # Simpan hasil visualisasi
    plt.tight_layout()
    plt.savefig(f"student_{student_id}_forecast.png")
    plt.close()


# Fungsi utama
def main():
    data, scaler = load_and_preprocess_data('student_monnitoring_data.csv')
    user_groups = data.groupby('Student ID')
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Proses hanya 5 mahasiswa sebagai contoh
    for i, (student_id, student_data) in enumerate(user_groups):
        # if i >= 5:
        #     break
        print(f"\nProcessing Hybrid for Student {student_id}")
        predict_hybrid(student_data, student_id, scaler, device)

In [2]:
# Hybrid ARIMA-LSTM dengan Output Per Tahap dan Batas 5 Student ID

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings("ignore")

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero = y_true != 0
    return np.mean(np.abs((y_true[non_zero] - y_pred[non_zero]) / y_true[non_zero])) * 100

def load_and_preprocess_data(file_path):
    data = pd.read_csv(file_path)
    data['Date'] = pd.to_datetime(data['Date'])
    data['Risk Level'] = data['Risk Level'].map({'Low': 0, 'Medium': 1, 'High': 2})
    data = data.dropna()

    # Pilih hanya kolom yang relevan
    selected_columns = ['Student ID', 'Date', 'Stress Level (GSR)', 'Sleep Hours', 'Anxiety Level', 'Mood Score', 'Risk Level']
    data = data[selected_columns]

    # Normalisasi fitur numerik
    numerical_features = ['Stress Level (GSR)', 'Sleep Hours', 'Anxiety Level', 'Mood Score']
    scaler = MinMaxScaler()
    data[numerical_features] = scaler.fit_transform(data[numerical_features])

    return data, scaler

def split_data_by_time(student_data, train_days=24):
    student_data = student_data.sort_values('Date')
    unique_dates = student_data['Date'].unique()
    train_dates = unique_dates[:train_days]
    test_dates = unique_dates[train_days:]
    train_data = student_data[student_data['Date'].isin(train_dates)]
    test_data = student_data[student_data['Date'].isin(test_dates)]
    return train_data, test_data

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.dropout = nn.Dropout(0.4)
        self.risk_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim // 2, num_classes)
        )
        self.fc_stress = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out, (h_n, _) = self.lstm(x)
        out = h_n[-1]
        out = self.bn(out)
        out = self.dropout(out)
        risk_logits = self.risk_head(out)
        stress = self.sigmoid(self.fc_stress(out))
        return risk_logits, stress.squeeze(1)

def load_arima_model(student_id, model_dir="model/arima"):
    model_path = os.path.join(model_dir, f"arima_student_{student_id}.pkl")
    if os.path.exists(model_path):
        with open(model_path, 'rb') as f:
            return pickle.load(f)
    raise FileNotFoundError(f"ARIMA model for Student {student_id} not found at {model_path}")

def load_lstm_model(student_id, input_dim, model_dir="model/lstm"):
    model_path = os.path.join(model_dir, f"lstm_student_{student_id}.pth")
    if os.path.exists(model_path):
        model = LSTMModel(input_dim=input_dim, hidden_dim=64, num_classes=3)
        model.load_state_dict(torch.load(model_path))
        return model
    raise FileNotFoundError(f"LSTM model for Student {student_id} not found at {model_path}")

def predict_hybrid_verbose(student_data, student_id, scaler, device):
    train_data, test_data = split_data_by_time(student_data)

    features = ['Stress Level (GSR)', 'Sleep Hours', 'Anxiety Level', 'Mood Score']
    target_cols = ['Risk Level', 'Stress Level (GSR)']

    X_train = train_data[features].values.reshape(-1, 1, len(features))
    X_test = test_data[features].values.reshape(-1, 1, len(features))
    y_train = train_data[target_cols].values
    y_test = test_data[target_cols].values

    arima_model = load_arima_model(student_id)
    arima_forecast = arima_model.forecast(steps=len(test_data))

    print(f"\n[Student {student_id}] ARIMA Forecast:")
    print(arima_forecast)

    X_train_aug = np.concatenate((X_train, np.zeros((len(train_data), 1, 1))), axis=2)
    X_test_aug = np.concatenate((X_test, arima_forecast.values.reshape(-1, 1, 1)), axis=2)

    X_test_tensor = torch.tensor(X_test_aug, dtype=torch.float32).to(device)
    model = load_lstm_model(student_id, input_dim=X_test_aug.shape[2])
    model.to(device)
    model.eval()

    with torch.no_grad():
        test_logits, test_stress = model(X_test_tensor)
        risk_pred_test = torch.argmax(test_logits, dim=1).cpu().numpy()
        stress_pred_test = test_stress.cpu().numpy()

    mae_stress = mean_absolute_error(y_test[:, 1], stress_pred_test)
    rmse_stress = np.sqrt(mean_squared_error(y_test[:, 1], stress_pred_test))
    mape_stress = mean_absolute_percentage_error(y_test[:, 1], stress_pred_test)
    mae_risk = mean_absolute_error(y_test[:, 0], risk_pred_test)
    acc_risk = np.mean(risk_pred_test == y_test[:, 0])

    print(f"\n[Student {student_id}] Evaluation:")
    print(f"MAE Stress: {mae_stress:.4f}, RMSE Stress: {rmse_stress:.4f}, MAPE Stress: {mape_stress:.2f}%")
    print(f"MAE Risk: {mae_risk:.4f}, Accuracy Risk: {acc_risk * 100:.2f}%")

    n_steps = 5
    window_input = X_test_aug[-n_steps:].copy()
    window_input = torch.tensor(window_input, dtype=torch.float32).to(device)
    future_preds = []

    arima_future = arima_model.forecast(steps=10).values
    arima_future = np.clip(arima_future, 0, 1)
    lstm_stress_series = []

    with torch.no_grad():
        for day in range(10):
            risk_logits_next, stress_pred_next = model(window_input)
            lstm_stress = stress_pred_next[-1].item()
            lstm_stress_series.append(lstm_stress)

            smoothed_stress = np.mean(lstm_stress_series[-3:]) if len(lstm_stress_series) >= 3 else lstm_stress
            alpha = np.clip(0.8 - (day * 0.05), 0.3, 0.8)
            combined_stress = alpha * smoothed_stress + (1 - alpha) * arima_future[day]
            combined_stress = np.clip(combined_stress + np.random.normal(0, 0.02), 0, 1)

            risk_level_pred = torch.argmax(risk_logits_next[-1]).item()
            last_input = window_input[-1, 0].cpu().numpy()
            next_features = np.clip(last_input[1:-1] + (np.random.rand(3) - 0.5) * 0.02, 0, 1)
            new_input = np.concatenate([[combined_stress], next_features, [arima_future[day]]])
            new_input_tensor = torch.tensor(new_input, dtype=torch.float32).reshape(1, 1, -1).to(device)
            window_input = torch.cat((window_input[1:], new_input_tensor), dim=0)
            future_preds.append([risk_level_pred, combined_stress])

    future_preds = np.array(future_preds)
    future_dates = pd.date_range(start=test_data['Date'].iloc[-1] + pd.Timedelta(days=1), periods=10)
    return stress_pred_test, risk_pred_test, future_preds, future_dates

def display_prediction_results(student_id, test_data, stress_pred_test, risk_pred_test, future_dates, future_preds):
    print(f"\n[Student {student_id}] Test Set Predictions:")
    df_test_result = test_data.copy()
    df_test_result['Predicted Stress'] = stress_pred_test
    df_test_result['Predicted Risk'] = risk_pred_test
    print(df_test_result[['Date', 'Stress Level (GSR)', 'Predicted Stress', 'Risk Level', 'Predicted Risk']])

    print(f"\n[Student {student_id}] 10-Day Forecast Results:")
    df_future = pd.DataFrame({
        'Date': future_dates,
        'Forecasted Stress': future_preds[:, 1],
        'Forecasted Risk': future_preds[:, 0].astype(int)
    })
    print(df_future)

def main():
    data, scaler = load_and_preprocess_data('student_monnitoring_data.csv')
    user_groups = data.groupby('Student ID')
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for i, (student_id, student_data) in enumerate(user_groups):
        if i >= 5:
            break
        print(f"\nProcessing Hybrid for Student {student_id}")
        train_data, test_data = split_data_by_time(student_data)

        print(f"\n[Student {student_id}] Train Data Sample:")
        print(train_data.head())

        print(f"\n[Student {student_id}] Test Data Sample:")
        print(test_data.head())

        stress_pred_test, risk_pred_test, future_preds, future_dates = predict_hybrid_verbose(
            student_data, student_id, scaler, device
        )

        display_prediction_results(student_id, test_data, stress_pred_test, risk_pred_test, future_dates, future_preds)

# Jalankan fungsi utama
main()  # Uncomment this line to run



Processing Hybrid for Student 1

[Student 1] Train Data Sample:
   Student ID       Date  Stress Level (GSR)  Sleep Hours  Anxiety Level  \
0           1 2024-12-01            0.093333        0.650       0.555556   
1           1 2024-12-02            0.148889        0.250       0.555556   
2           1 2024-12-03            0.902222        0.325       0.333333   
3           1 2024-12-04            0.571111        1.000       0.111111   
4           1 2024-12-05            0.762222        0.600       0.888889   

   Mood Score  Risk Level  
0    0.555556           0  
1    0.111111           1  
2    0.777778           2  
3    1.000000           0  
4    0.333333           2  

[Student 1] Test Data Sample:
    Student ID       Date  Stress Level (GSR)  Sleep Hours  Anxiety Level  \
24           1 2024-12-25            0.433333        0.400       0.555556   
25           1 2024-12-26            0.533333        0.550       1.000000   
26           1 2024-12-27            0.308889   


[Student 4] Evaluation:
MAE Stress: 0.1183, RMSE Stress: 0.1446, MAPE Stress: 63.78%
MAE Risk: 0.1667, Accuracy Risk: 83.33%

[Student 4] Test Set Predictions:
          Date  Stress Level (GSR)  Predicted Stress  Risk Level  \
114 2024-12-25            0.075556          0.249494           1   
115 2024-12-26            0.988889          0.985070           2   
116 2024-12-27            0.968889          0.984905           2   
117 2024-12-28            0.166667          0.338177           0   
118 2024-12-29            0.675556          0.903791           2   
119 2024-12-30            0.844444          0.960630           2   

     Predicted Risk  
114               2  
115               2  
116               2  
117               0  
118               2  
119               2  

[Student 4] 10-Day Forecast Results:
        Date  Forecasted Stress  Forecasted Risk
0 2024-12-31           0.971752                2
1 2025-01-01           0.947975                2
2 2025-01-02           